In [1]:
import scipy.sparse as sp
import numpy as np
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LinearRegression, Ridge
import time
import random, sys

In [2]:
ratings = np.load("ratings.npy")

In [3]:
number_of_users = len(np.unique(ratings[:,0]))
number_of_restaurants = len(np.unique(ratings[:,1]))
M = sp.csr_matrix((ratings[:,2], (ratings[:,0], ratings[:,1])), 
                  shape=(number_of_users, number_of_restaurants))
M

<337867x5899 sparse matrix of type '<class 'numpy.uint32'>'
	with 929606 stored elements in Compressed Sparse Row format>

In [17]:
def cold_start_preprocessing(matrix, min_entries):
    """
    Recursively removes rows and columns from the input matrix which have less than min_entries nonzero entries.
    
    Parameters
    ----------
    matrix      : sp.spmatrix, shape [N, D]
                  The input matrix to be preprocessed.
    min_entries : int
                  Minimum number of nonzero elements per row and column.

    Returns
    -------
    matrix      : sp.spmatrix, shape [N', D']
                  The pre-processed matrix, where N' <= N and D' <= D
        
    """
    print("Shape before: {}".format(matrix.shape))
    
    ### YOUR CODE HERE ###
    def remove_users_restaurants(matrix):
        shape = matrix.shape
        matrix = matrix[matrix.getnnz(1)>min_entries]
        matrix = matrix[:,matrix.getnnz(0)>min_entries]
        shape_new = matrix.shape
        if shape == shape_new:
            return matrix
        else:
            return remove_users_restaurants(matrix)
    matrix = remove_users_restaurants(matrix)
    print("Shape after: {}".format(matrix.shape))
    nnz = matrix>0
    assert (nnz.sum(0).A1 > min_entries).all()
    assert (nnz.sum(1).A1 > min_entries).all()
    return matrix

In [18]:
M = cold_start_preprocessing(M, 10)

Shape before: (337867, 5899)
Shape after: (11275, 3531)
